In [1]:
import os
import time

import pandas as pd
import requests
from tqdm import tqdm

In [2]:
usernames = ["Fro116", "mpfei"]

In [3]:
os.chdir("../../data/user_profiles")

In [4]:
def get_score_df(json):
    # turn the json response into a dataframe
    anime_ids = []
    scores = []
    for item in response.json()["anime"]:
        anime_ids.append(item["mal_id"])
        scores.append(item["score"])
    return pd.DataFrame({"anime_id": anime_ids, "my_score": scores})

In [5]:
user_lists = []

max_items_per_page = 300  # property of the API
for username in tqdm(usernames):
    more_pages = True
    page = 1
    scores = []
    while more_pages:
        time.sleep(5)
        response = requests.get(
            f"https://api.jikan.moe/v3/user/{username}/animelist/all/{page}"
        )
        page_df = get_score_df(response.json())
        scores.append(page_df)
        page += 1
        if len(page_df) < max_items_per_page:
            more_pages = False
    score_df = pd.concat(scores)
    score_df["username"] = username
    score_df = score_df.loc[lambda x: x["my_score"] != 0]
    user_lists.append(score_df)

100%|██████████| 2/2 [00:19<00:00,  9.68s/it]


In [6]:
scores = pd.concat(user_lists)

In [9]:
scores.to_pickle("ExtraUserAnimeLists.pkl")

In [8]:
scores

,anime_id,my_score,username
0,7193,7,Fro116
1,820,10,Fro116
3,24629,6,Fro116
5,31646,7,Fro116
6,35180,7,Fro116
...,...,...,...
74,25519,8,mpfei
76,1050,8,mpfei
77,245,8,mpfei
78,38483,8,mpfei
